In [ ]:
# learned the basics of this from https://www.youtube.com/watch?v=Mz9JG9CUXXY
# gsheets api key is at https://console.cloud.google.com/apis/credentials/key/74ab9316-bec9-46fe-bea0-39ced5c4ecdf?inv=1&invt=Abl-Wg&project=c46songdatagetter
# had to make the G sheet public to avoid needing a service account...may want to do that later

# I'm using .venv Python 3.10.8 kernel

import requests
import os
from googleapiclient.discovery import build
# from google.cloud import language
from dotenv import load_dotenv

load_dotenv()

In [2]:
API_KEY = str(os.getenv('GSHEET_API_KEY'))
SPREADSHEET_ID = str(os.getenv('C46_GSHEET_ID'))
RANGE_NAME = 'Filtered!A1:M108'

In [3]:
def authenticate_sheets(api_key):
    return build('sheets', 'v4', developerKey=api_key).spreadsheets()

In [4]:
sheets = authenticate_sheets(API_KEY)
result = sheets.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()

gSheet_data = result['values']
keys = gSheet_data[0]
values = gSheet_data[1:]

In [5]:
# songs = dict.fromkeys(keys)
gSheet_songs = []

for value in values:
    # value[1].replace('\xa0','')
    gSheet_songs.append(dict(zip(keys, value)))

In [ ]:
print(type(gSheet_songs[5]['Song Title']))

print(gSheet_songs[5]['Song Title'])

In [ ]:
gSheet_songs

In [8]:
import pandas as pd

songs_df = pd.DataFrame(gSheet_songs)

In [ ]:
songs_df

In [ ]:
song_titles = []

for song in values:
    song_titles.append(song[1])

print(song_titles)

In [11]:
import Songs

In [ ]:
mySongs_df = Songs.getSongs()
mySongs = []

for song in mySongs_df.itertuples():
    mySongs.append((song.artist, song.title))
    # mySongTuples.append((song['artist'],song['title']))

print(mySongs)

In [13]:
sbp_songs = sorted(mySongs, key=lambda x: x[1])

In [ ]:
sbp_songs

In [15]:
import fuzzyset as fs

song_set = fs.FuzzySet(song_titles)

In [ ]:
song_set.get('7+7')

In [17]:
song_matches = []

for song in sbp_songs:
    # song_match = song_set.get(song)[0] + (song,)
    # song_matches.append(song_match)
    song_matches.append(dict(sbpArtist=song[0], sbpSong=song[1], matchFit=song_set.get(song[1])[0][0], gSheetSong=song_set.get(song[1])[0][1]))

song_matches_df = pd.DataFrame(song_matches)

In [ ]:
song_matches_df.query('matchFit < .5')

In [ ]:
song_matches_df.query('sbpSong.str.contains("Oklahoma")')

In [ ]:
sorted_songs = []

for song in gSheet_songs:
    sorted_songs.append((song['Song Title'].replace(' \xa0','').replace('\xa0','')))

sorted_songs.sort()

sorted_songs